# Sentiment Analysis POC

* This notebook is a scratch pad, for sentiment analysis POC. 
* The idea is to try out few pre-trained sentiment analysis models and see which one works for our use case. 

## Installations

In [65]:
# # ## installing required libraries
# ! pip install beautifulsoup4
# ! pip install pandas
# ! pip install numpy
# ! pip install plotly
# ! pip install nbformat
# ! pip install ipykernel
# ! pip install matplotlip
# ! pip install wordcloud
# ! pip install gensim
# ! pip install pyLDAvis
# ! pip install nltk
# ! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_trf 
# ! python -m spacy download en_core_web_md
# ! pip install joblib
# ! pip install tqdm
# ! pip install transformers


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Data

In [2]:
## lets load 
import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

import re
import string
from bs4 import BeautifulSoup

import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
from pprint import pprint

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

# loading library
import pickle

from joblib import dump, load

from tqdm.auto import tqdm

import os
import sys
sys.path.insert(0, os.path.abspath('../utils'))

## importing custom modules
import common_utils
import gensim_utils
import sklearn_utils

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\workspace\Aletheia\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\workspace\Aletheia\env\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] D

## Reading Data

In [3]:
## reading manaully scrapped data
data = pd.read_csv('../data/scrapped_fox_data_clean.csv')
print(data.shape)

(3972, 12)


In [4]:
def preprocess_text(text):
     text = common_utils.clean_html(text)
     text = common_utils.lower_case(text)
     text = common_utils.remove_line_breaks(text)
     text = common_utils.remove_punctuation(text)
     text = common_utils.remove_numbers(text)
     text = common_utils.remove_extra_spaces(text)
     # text = remove_stopwords(text)
     return text

data["clean_text"] = data["text"].progress_apply(preprocess_text)

processing: 100%|██████████| 3972/3972 [00:01<00:00, 2542.73it/s]


In [5]:
data.head()["clean_text"]

0    former governor and first term democratic sen ...
1    president biden urged democrats on wednesday t...
2    the famous naked cowboy in new york citys time...
3    liberal groups in wisconsin seeking to change ...
4    texas gubernatorial nominee beto o’rourke is a...
Name: clean_text, dtype: object

## Sentiment Analysis Using VADER
(Valence Aware Dictionary and sEntiment Reasoner)

##### Notes
* Uses Bag of words approach
* Gives +tive, -tive or neutral values to each of the words in the sentence and then gives combined value of that to tell us whether the sentence is positive, negative or neutral
* Does not account for relationship between words :(

In [6]:
from nltk.sentiment import SentimentIntensityAnalyzer


In [7]:
example = data["text"][0]
example

'Former governor and first term Democratic Sen. Maggie Hassan of New Hampshire and Republican challenger Don Bolduc took aim at each other over inflation, abortion, national security, the border crisis, election denialism, and many more issues in their third and final debate in their crucial battleground state race that’s among a handful across the country that will likely determine if the GOP wins back the Senate majority. But ahead of the verbal crossfire on the debate stage, Bolduc – a former Army general who served ten tours of duty in the war in Afghanistan – was allegedly assaulted as he arrived at the debate site at Saint Anselm College’s New Hampshire Institute of Politics on Wednesday evening. According to the Bolduc campaign, a bystander standing in the crowd outside the debate site took a swing at the former general as he arrived. The campaign says Bolduc was slightly grazed but not injured.&nbsp; Rick Wiley of the Bolduc campaign tells Fox News the person who threw the punc

In [8]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
sia = SentimentIntensityAnalyzer()

In [16]:
sia.polarity_scores(example)

{'neg': 0.105, 'neu': 0.812, 'pos': 0.083, 'compound': -0.9869}

In [21]:
## running the sentiment analysis on entire dataset
print(data["title"][0])
sia.polarity_scores(data["title"][0])

Hassan and Bolduc trade fire in final showdown after GOP nominee comes under attack arriving at debate


{'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'compound': -0.6705}

In [27]:
## lets rename index to id
data.rename(columns={"index":"id"}, inplace=True)

In [57]:
results = {}
for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
    text = row["title"]
    id = idx
    results[id] = sia.polarity_scores(text)    

100%|██████████| 3972/3972 [00:00<00:00, 5999.72it/s]


In [58]:
results_df = pd.DataFrame.from_dict(results, orient='index')

In [59]:
combined_df = pd.concat([data, results_df], axis=1)
combined_df.head()

,title,description,url,last_published_date,authors,text,published_day,published_month,num_authors,author,word_count,line_count,clean_text,neg,neu,pos,compound
0,Hassan and Bolduc trade fire in final showdown...,A bystander took a swing at Republican Senate ...,https://www.foxnews.com/politics/hassan-bolduc...,2022-11-02 22:47:00-04:00,[{'name': 'Paul Steinhauser'}],Former governor and first term Democratic Sen....,2,11,1,Paul_Steinhauser,1271,62,former governor and first term democratic sen ...,0.268,0.732,0.000,-0.6705
1,Biden suggests voting for Republicans is a thr...,President Biden said the only way to repudiate...,https://www.foxnews.com/politics/biden-speech,2022-11-02 22:15:46-04:00,[{'name': 'Haris Alic'}],President Biden urged Democrats on Wednesday t...,2,11,1,Haris_Alic,478,22,president biden urged democrats on wednesday t...,0.298,0.702,0.000,-0.5267
2,NYC's Naked Cowboy makes endorsement for gov w...,New York City's Naked Cowboy endorsed Lee Zeld...,https://www.foxnews.com/politics/nyc-naked-cow...,2022-11-02 21:58:25-04:00,[{'name': 'Adam Sabes'}],The famous Naked Cowboy in New York City's Tim...,2,11,1,Adam_Sabes,205,18,the famous naked cowboy in new york citys time...,0.000,0.757,0.243,0.5423
3,Wisconsin courts shoot down liberal groups' at...,A Wisconsin appeals court and a circuit judge ...,https://www.foxnews.com/politics/wisconsin-cou...,2022-11-02 21:44:40-04:00,[{'name': 'Bradford Betz'}],Liberal groups in Wisconsin seeking to change ...,2,11,1,Bradford_Betz,381,20,liberal groups in wisconsin seeking to change ...,0.290,0.710,0.000,-0.5423
4,Texas gubernatorial candidate Beto O'Rourke jo...,Texas gubernatorial nominee Beto O’Rourke is t...,https://www.foxnews.com/politics/texas-guberna...,2022-11-02 20:38:30-04:00,[{'name': 'Bradford Betz'}],Texas gubernatorial nominee Beto O’Rourke is a...,2,11,1,Bradford_Betz,267,15,texas gubernatorial nominee beto o’rourke is a...,0.000,1.000,0.000,0.0000


In [63]:
combined_df.loc[combined_df["pos"].idxmax(), "title"]

"Former President Trump celebrates 'ALL' endorsement wins in primary: 'Great candidates!'"

In [64]:
combined_df.describe()

,published_day,published_month,num_authors,word_count,line_count,neg,neu,pos,compound
count,3972.000000,3972.000000,3972.000000,3972.000000,3972.000000,3972.000000,3972.000000,3972.000000,3972.000000
mean,16.787513,8.453424,1.119084,610.804884,31.652064,0.109221,0.812165,0.078617,-0.065043
std,8.934083,1.335676,0.447909,340.785100,19.232276,0.136235,0.158318,0.110806,0.394567
min,1.000000,6.000000,0.000000,31.000000,3.000000,0.000000,0.163000,0.000000,-0.966100
25%,9.000000,7.000000,1.000000,399.000000,21.000000,0.000000,0.701000,0.000000,-0.361200
50%,18.000000,8.000000,1.000000,534.000000,28.000000,0.000000,0.823000,0.000000,0.000000
75%,25.000000,10.000000,1.000000,734.000000,37.000000,0.194000,1.000000,0.155000,0.177900
max,31.000000,11.000000,5.000000,9672.000000,647.000000,0.837000,1.000000,0.668000,0.936000


## Roberta Model

In [90]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax

In [91]:
## pulling a specific model pretrained on sentiment analysis
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [92]:
print(data["title"][1])
sia.polarity_scores(data["title"][1])

Biden suggests voting for Republicans is a threat to democracy


{'neg': 0.298, 'neu': 0.702, 'pos': 0.0, 'compound': -0.5267}

In [93]:
## running on roberta model
def get_roberta_sentiment(text):
    encoded_text = tokenizer(text, return_tensors="pt")
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ## scores are in order of negative, neutral and positive
    scores_dict = {"roberta_neg":scores[0], "roberta_neu":scores[1], "roberta_pos":scores[2]}
    return scores_dict


scores = get_roberta_sentiment(data["title"][1])
scores

{'roberta_neg': 0.66241485,
 'roberta_neu': 0.3247515,
 'roberta_pos': 0.012833751}

In [109]:
results = {}
for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
    try:
        text = row["text"]
        id = idx
        vader_results = sia.polarity_scores(text)
        vader_results_rename = {}
        for k,v in vader_results.items():
            vader_results_rename[f"vader_{k}"] = v
        roberta_results = get_roberta_sentiment(text)
        both = {**vader_results_rename, **roberta_results}
        results[idx] = both
    except RuntimeError:
        print(f"Broke for id {idx}")


  0%|          | 0/3972 [00:00<?, ?it/s]

Broke for id 0
Broke for id 1


  0%|          | 6/3972 [00:02<37:46,  1.75it/s]

Broke for id 6


  0%|          | 15/3972 [00:03<09:30,  6.93it/s]

...

Broke for id 95
Broke for id 96


  2%|▏         | 98/3972 [00:23<16:53,  3.82it/s]

Broke for id 98


  2%|▏         | 99/3972 [00:24<15:39,  4.12it/s]


KeyboardInterrupt: 

In [95]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos
0,0.268,0.732,0.000,-0.6705,0.214919,0.765167,0.019915
1,0.298,0.702,0.000,-0.5267,0.662415,0.324751,0.012834
2,0.000,0.757,0.243,0.5423,0.005948,0.538470,0.455582
3,0.290,0.710,0.000,-0.5423,0.346270,0.637059,0.016671
4,0.000,1.000,0.000,0.0000,0.056946,0.885563,0.057491
...,...,...,...,...,...,...,...
3967,0.152,0.691,0.157,0.0258,0.224884,0.703206,0.071910
3968,0.000,1.000,0.000,0.0000,0.570033,0.403358,0.026609
3969,0.080,0.747,0.172,0.3818,0.502582,0.481971,0.015446
3970,0.173,0.827,0.000,-0.3182,0.048049,0.920026,0.031925


In [102]:
final_df = pd.concat([data, results_df], axis=1)


In [105]:
final_df.loc[final_df["roberta_pos"].idxmax(), "title"]

"Crist praises Biden, says president is 'phenomenal' and he 'can't wait' to have his support in Florida"

In [107]:
final_df.loc[final_df["vader_pos"].idxmax(), "title"]

"Former President Trump celebrates 'ALL' endorsement wins in primary: 'Great candidates!'"